In [2]:
import json
import unicodedata
import string
import os

def remover_acentos(texto: str) -> str:
    """Remove acentos de uma string."""
    nfkd_form = unicodedata.normalize('NFKD', texto)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def remover_pontuacao(texto: str) -> str:
    """Remove pontuação de uma string."""
    translator = str.maketrans('', '', string.punctuation)
    return texto.translate(translator)

def normalizar_texto(texto: str) -> str:
    """Normaliza um texto: minúsculas, sem acentos, sem pontuação e remove espaços extras."""
    if not isinstance(texto, str):
        return texto

    texto = texto.lower()
    texto = remover_acentos(texto)
    texto = remover_pontuacao(texto)
    texto = " ".join(texto.split())
    return texto

def normalizar_estrutura_json(data):
    """
    Navega recursivamente pela estrutura JSON (dicionários e listas)
    e aplica a normalização de texto a todas as chaves de string e valores de string.
    """
    if isinstance(data, dict):
        novo_dict = {}
        for k, v in data.items():
            chave_normalizada = k # Por padrão, assume que a chave pode não ser string
            if isinstance(k, str):
                chave_normalizada = normalizar_texto(k)
            
            valor_normalizado = normalizar_estrutura_json(v)
            
            # Se a chave normalizada já existe e o valor é diferente,
            # você pode querer uma estratégia de mesclagem ou aviso.
            # Por simplicidade, a última entrada com a mesma chave normalizada prevalecerá.
            if chave_normalizada in novo_dict and novo_dict[chave_normalizada] != valor_normalizado:
                print(f"Aviso: Chave normalizada duplicada '{chave_normalizada}' encontrada. "
                      f"Valor antigo: '{novo_dict[chave_normalizada]}', "
                      f"Novo valor: '{valor_normalizado}'. O novo valor prevalecerá.")
            novo_dict[chave_normalizada] = valor_normalizado
        return novo_dict
    elif isinstance(data, list):
        return [normalizar_estrutura_json(item) for item in data]
    elif isinstance(data, str):
        return normalizar_texto(data)
    else:
        return data

def main():
    # Caminho do arquivo de entrada (usando o arquivo que você gerou como exemplo de entrada agora)
    # Se quiser rodar no original, mude para setor_map.json
    caminho_arquivo_entrada = r"C:\Users\MENICIO JR\Desktop\Natural2SPARQL-master\src\main\resources\setor_map.json"
    # Ou, se você quiser processar o arquivo que já tinha valores normalizados:
    # caminho_arquivo_entrada = r"C:\Users\MENICIO JR\Desktop\Natural2SPARQL-master\src\main\resources\setor_map_normalizado.json"


    if not os.path.exists(caminho_arquivo_entrada):
        print(f"Erro: O arquivo de entrada não foi encontrado em '{caminho_arquivo_entrada}'")
        return

    diretorio_entrada, nome_arquivo_entrada = os.path.split(caminho_arquivo_entrada)
    nome_base, extensao = os.path.splitext(nome_arquivo_entrada)
    # Nome do arquivo de saída para evitar sobrescrever o anterior se você rodar múltiplas vezes
    caminho_arquivo_saida = os.path.join(diretorio_entrada, f"{nome_base}_totalmente_normalizado{extensao}")

    try:
        with open(caminho_arquivo_entrada, 'r', encoding='utf-8') as f_entrada:
            dados_json = json.load(f_entrada)
        print(f"Arquivo '{caminho_arquivo_entrada}' lido com sucesso.")

        dados_normalizados = normalizar_estrutura_json(dados_json)
        print("Dados (chaves e valores) normalizados.")

        with open(caminho_arquivo_saida, 'w', encoding='utf-8') as f_saida:
            json.dump(dados_normalizados, f_saida, ensure_ascii=False, indent=4)
        print(f"Arquivo totalmente normalizado salvo em '{caminho_arquivo_saida}'")

    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em '{caminho_arquivo_entrada}'")
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{caminho_arquivo_entrada}' não é um JSON válido.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

if __name__ == "__main__":
    main()

Arquivo 'C:\Users\MENICIO JR\Desktop\Natural2SPARQL-master\src\main\resources\setor_map.json' lido com sucesso.
Dados (chaves e valores) normalizados.
Arquivo totalmente normalizado salvo em 'C:\Users\MENICIO JR\Desktop\Natural2SPARQL-master\src\main\resources\setor_map_totalmente_normalizado.json'
